The aim of this script is to simulate the motion a chaser satellite in close proximity to a target satellite. This uses the Clohessy Wiltshire equations to draw the chaser satellite in the reference frame of the target satellite. This particular simulation attempts to simulate the motion of the satellite given initial conditions and then provides the means to apply an extended burn across a chosen time period and fuel capacity.

In [ ]:
import numpy as np
import plotly.graph_objects as go

# Gravitational parameter (mu) and mean motion (n)
mu = 398600.4418  # km^3/s^2 (gravitational parameter of Earth)
sma = 7000  # km (semi-major axis of the target's orbit)
n = np.sqrt(mu / sma**3)  # mean motion of the target

# Initial conditions in the target's orbit reference frame
r0_chaser = np.array([100000.0, 9000000.0, -3000000.0])  # Initial relative position of chaser, km
v0_chaser = np.array([30000, 500, 0.1])  # Initial relative velocity of chaser, km/s

# Thrust parameters
thrust_start_time = 2000  # Time delay before thrust starts in seconds
thrust_duration = 60    # Thrust duration in seconds
thrust_force = np.array([-10000, 0, 0])  # Thrust force in km/s^2
fuel = 1000000  # Initial fuel amount

# Time parameters
t_final = 50000  # Total simulation time, seconds
dt = 1  # Time step, seconds

# Time array
t = np.arange(0, t_final, dt)

# Position and velocity arrays
r_chaser = np.zeros((len(t), 3))
v_chaser = np.zeros((len(t), 3))

# Set initial conditions
r_chaser[0] = r0_chaser
v_chaser[0] = v0_chaser

# Clohessy-Wiltshire equations for relative motion
def clohessy_wiltshire(r, v, t, n):
    x, y, z = r
    x_dot, y_dot, z_dot = v
    
    x_ddot = 3 * n**2 * x + 2 * n * y_dot
    y_ddot = -2 * n * x_dot
    z_ddot = -n**2 * z
    
    return np.array([x_dot, y_dot, z_dot]), np.array([x_ddot, y_ddot, z_ddot])

# Simulation loop
for i in range(1, len(t)):
    # Get current state
    r = r_chaser[i-1]
    v = v_chaser[i-1]
    
    # Find accelerations using Clohessy-Wiltshire equations
    v, a = clohessy_wiltshire(r, v, t[i], n)
    
    # Apply thrust if within thrust duration and fuel is available
    if thrust_start_time <= t[i] <= (thrust_start_time + thrust_duration) and fuel > 0:
        thrust_acceleration = thrust_force  # Thrust acceleration in km/s^2
        fuel -= np.linalg.norm(thrust_force) * dt  # Decrease fuel
        print(fuel)
        fueldepleted = t[i]
    else:
        thrust_acceleration = np.array([0.0, 0.0, 0.0])  # No thrust

    # Total acceleration
    total_acceleration = a + thrust_acceleration

    # Update position and velocity
    r_chaser[i] = r + v * dt
    v_chaser[i] = v + total_acceleration * dt

# Plotting with Plotly for interactive visualization
r_chaser_scaled=np.divide(r_chaser,sma)

# Create a trace for the chaser
trace = go.Scatter3d(
    x=r_chaser_scaled[:, 0],
    y=r_chaser_scaled[:, 1],
    z=r_chaser_scaled[:, 2],
    mode='markers',
    name='Chaser Satellite',
    marker=dict(
        color='blue',
        size=2
    )
)

#create marker for inital position
initial_trace = go.Scatter3d(
    x=[r_chaser_scaled[0, 0]],
    y=[r_chaser_scaled[0, 1]],
    z=[r_chaser_scaled[0, 2]],
    mode='markers',
    name='Initial Position',
    marker=dict(
        color='black',
        size=5
    )
)

# Create a trace for the target
target_trace = go.Scatter3d(
    x=[0],
    y=[0],
    z=[0],
    mode='markers',
    name='Target Satellite',
    marker=dict(
        color='red',
        size=5
    )
)

# Create marker for start of thrust
thrust_start_trace = go.Scatter3d(
    x=[r_chaser_scaled[thrust_start_time, 0]],
    y=[r_chaser_scaled[thrust_start_time, 1]],
    z=[r_chaser_scaled[thrust_start_time, 2]],
    mode='markers',
    name='Thrust Start',
    marker=dict(
        color='green',
        size=5
    )
)

# Create marker for end of thrust
if fuel <= 0:
    thrust_end_trace = go.Scatter3d(
        x=[r_chaser_scaled[fueldepleted, 0]],
        y=[r_chaser_scaled[fueldepleted, 1]],
        z=[r_chaser_scaled[fueldepleted, 2]],
        mode='markers',
        name='Thrust End',
        marker=dict(
            color='yellow',
            size=5
        )
    )
else:
    thrust_end_trace = go.Scatter3d(
        x=[r_chaser_scaled[thrust_start_time+thrust_duration, 0]],
        y=[r_chaser_scaled[thrust_start_time+thrust_duration, 1]],
        z=[r_chaser_scaled[thrust_start_time+thrust_duration, 2]],
        mode='markers',
        name='Thrust End',
        marker=dict(
            color='yellow',
            size=5
        )
    )

# Create the layout
layout = go.Layout(
    title='Relative Motion of the Chaser Satellite',
    scene=dict(
        xaxis=dict(title='X/a [km]'),
        yaxis=dict(title='Y/a [km]'),
        zaxis=dict(title='Z/a [km]')
    )
)

# Create the figure and add the traces
fig = go.Figure(data=[trace, target_trace, thrust_start_trace, initial_trace, thrust_end_trace], layout=layout)

# Show the figure
fig.show()